In [ ]:
import numpy as np
from scipy.stats import ortho_group
import sklearn.linear_model as lin
import time
from mpc_svrg import * # make sure to load the version which calls sag everywhere

In [ ]:
# check performance
def tau_lmax(A):
    evals = np.linalg.eig(np.dot(A.T, A))[0]
    return (np.sum(evals) / np.min(evals), np.max(evals))

def tau(A):
    evals = np.linalg.eig(np.dot(A.T, A))[0]
    return np.sum(evals) / np.min(evals)

def WhalfA(A, w):
    whalfscale = np.reshape(np.sqrt(w), (np.shape(A)[0], 1))
    return A * whalfscale

In [ ]:
def subspace_basis(t, d):
    U = ortho_group.rvs(d)
    return U[:t]

# B is t x d orthogonal basis vectors, project it out of vector v
def subspace_project(B, v):
    for i in xrange(len(B)):
        v = v - np.dot(v, B[i]) * B[i]
    return v

# scale covariance down
def scaled_normal(d):
    return np.random.normal(scale = 1.0 / np.sqrt(d), size = d)

In [ ]:
# random n1 rows outside small subspace
# basis of subspace, blown up to be quite large
# other n2 random rows that are also large
def bad_instance(t, d, n1, n2, size):
    B = subspace_basis(t, d)
    A = np.asarray([subspace_project(B, scaled_normal(d)) for i in xrange(n1)])
    C = np.asarray([scaled_normal(d) for i in xrange(n2)])
    D = np.vstack((A, B * size, C * size))
    np.random.shuffle(D)
    return D

# SAG fails to converge

In [ ]:
(t, d, n1, n2, size) = (10, 200, 1000, 100, 200)
A = bad_instance(t, d, n1, n2, size)
xtrue = np.random.normal(size = d)
b = np.dot(A, xtrue)
print xtrue[:20]
print tau(A)

In [ ]:
reg = lin.Ridge(alpha=0.0, solver='sag', tol=1e-11)
start = time.time()
x = reg.fit(A, b).coef_
print x[:20]
end = time.time()
print end - start

# Preprocessing via MPC converges

In [ ]:
degree = 10 # degree of rational approximation
jldirs = 5 # number of JL directions used
niters = 7 # number of iterations of solver
alpha = 1.0 # step size of solver

coeffs = np.asarray(ratapprox(degree))
start = time.time()
w = mpc_ideal_niters(A, 800, alpha, niters, jldirs, coeffs)
x = reg.fit(WhalfA(A, w), np.sqrt(w) * b).coef_
print x[:20]
end = time.time()
print end - start
print taufull(AWA(A, w))